In [ ]:
import requests
import time

named_tuple = time.localtime() # get struct_time
time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
print(time_string)

In [ ]:
BOT_TOKEN =''
CHAT_ID = 5686142393
seuil = 55
once = True

In [ ]:
def send_message(msg):
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    params = { "chat_id": CHAT_ID, "text": msg }
    requests.get(url, params=params)

def telegram(pi, temp):
    global once
    if temp > seuil and once:
        named_tuple = time.localtime() # get struct_time
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
        send_message(f"{time_string} {pi} en surchauffe : {temp}")
        once = False

#telegram('pi4', 60)
#send_message(" Crawling progress: 56% done")

In [ ]:
import paho.mqtt.client as mqtt
# topic to publish on broker
topic = "cpu_temp"
temps = [[]]*3
broker1 = "192.168.0.175"
broker2 = "192.168.0.184"

In [ ]:
def on_connect(client, userdata, flags, rc):
    # subscribe
    client.subscribe(topic)
    #clrshell()

def on_message(client, userdata, msg):
    global once
    #print(f"{msg.topic} {msg.payload}")
    temp = msg.payload.decode('utf8').split(' : ')[1]
    if 'Pi4' in temp:
        temps[0] = round(float(temp.split('°')[0]),2)
        if temps[0]<seuil:
            once=True
        telegram('Pi4',temps[0])
    if 'Pi3' in temp:
        temps[2] = round(float(temp.split('°')[0]),2)
        #telegram('Pi4',temps[2])
    else:
        temps[2] = 'None'
    if 'Zero' in temp:
        temps[1] = round(float(temp.split('°')[0]),2)
        if temps[1]<seuil:
            once=True
        telegram('Pi0',temps[1])
    
    roll = False
    
    if roll:
        # roll with show plotter
        print('Pi4', temps[0], 'Pi Zero', temps[1], 'Pi3', temps[2])
    else:
        # one line but show plotter does not work 
        print('\rPi4 :', temps[0],end=' - ')
        print('Pi Zero :', temps[1],end=' - ')
        print('Pi3 :', temps[2],end='           ')

client = mqtt.Client()
client.connect(broker2, 1883, 60)
client.on_connect = on_connect
client.on_message = on_message
   
client.loop_forever()